<a href="https://colab.research.google.com/github/pachterlab/LSCHWCP_2023/blob/main/Notebooks/Figure_7/viral_profiles_per_celltype.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Viral profiles across host clusters and cell types

In [ ]:
!pip install -q anndata
import anndata
import numpy as np
import random
import pandas as pd
import json
import os
import glob
import matplotlib.pyplot as plt
%config InlineBackend.figure_format='retina'

def nd(arr):
    """
    Function to transform numpy matrix to nd array.
    """
    return np.asarray(arr).reshape(-1)

def flatten(xss):
    """
    Function to flatten array of arrays.
    """
    return [x for xs in xss for x in xs]

___
# Load data
Count matrices generated [here](https://github.com/pachterlab/LSCHWCP_2023/blob/main/Notebooks/Supp_Fig_3/Supp_Fig_3d/1_macaque_celltype_assignment.ipynb) (host) and [here](https://github.com/pachterlab/LSCHWCP_2023/tree/main/Notebooks/align_macaque_PBMC_data/7_virus_host_captured_dlist_cdna_dna) (viral).

In [ ]:
# Download count matrices from Caltech data




In [ ]:
# Macaque only
host_adata = anndata.read("macaque_QC_norm_leiden_celltypes.h5ad")
host_adata

In [ ]:
palmdb_adata = anndata.read("virus_host-captured_dlist_cdna_dna.h5ad")
palmdb_adata

Load ID to phylogeny mapping:

In [ ]:
# Load virus ID to taxonomy mapping
!wget https://raw.githubusercontent.com/pachterlab/LSCHWCP_2023/main/PalmDB/ID_to_taxonomy_mapping.csv
id2tax = pd.read_csv("ID_to_taxonomy_mapping.csv")
id2tax

___
# EBOV counts per timepoint

In [ ]:
def rand_jitter(arr, jitter_size=0.02):
    """
    Add random jitter to array to avoid overplotting.
    """
    return np.random.uniform(low=-jitter_size, high=jitter_size, size=(len(arr),))

Group timepoints:

In [ ]:
palmdb_adata.obs["dpi_grouped"] = palmdb_adata.obs["dpi_clean_merged"]
dpi_dict = {"-30d":"pre-infection", "-4d":"pre-infection", "0d":"pre-infection", "4h":"4h", "24h":"24h", "3d":"early", "4d":"middle", "5d":"middle", "6d":"late", "7d/8d":"late"}

palmdb_adata.obs = palmdb_adata.obs.replace({"dpi_grouped": dpi_dict})

In [ ]:
fig, axs = plt.subplots(figsize=(9, 9), nrows=2, sharex=True)
fontsize = 18
width = 0.6
color = "#003049" # Change to #b266b3 for black bkg
background = "white"
label_color = "black"
grid_color = "lightgrey"

samples = ["pre-infection", "4h", "24h", "early", "middle", "late"]
labels = ["Baseline\n(-30-0d)", "Incubation\n(4h)", "Incubation\n(24h)", "Early\n(3d)", "Middle\n(4-5d)", "Late\n(6-8d)"]
target_ids = id2tax[id2tax["species"].str.contains("Zaire ebolavirus")]["rep_ID"].unique()

# Plot number of ZEBOV counts per timepoint
ax = axs[0]

cellnum = []
total_num_cells = []
for sample in samples:
    total = len(palmdb_adata.obs[(palmdb_adata.obs["celltype"].notnull()) & (palmdb_adata.obs["dpi_grouped"] == sample)])
    total_num_cells.append(total)

    pos_cells = palmdb_adata[(palmdb_adata.obs["celltype"].notnull()) & (palmdb_adata.obs["dpi_grouped"] == sample), palmdb_adata.var.index.isin(target_ids)].X.sum()
    cellnum.append((pos_cells / total) * 10000)

    print(f"Total number of positive cells in sample '{sample}': {pos_cells}")

x = np.arange(len(labels))

ax.bar(
    x,
    cellnum,
    width=width,
    # label=f"EBOV+ cells",
    color=color,
    alpha=1,
    edgecolor=label_color,
)

# Add total number of cells per condition
for x_idx, nc in enumerate(total_num_cells):
    ax.text(x_idx, 75, "{:,}".format(nc), color=label_color, fontsize=fontsize, va="bottom", ha="center")

ax.set_yscale("symlog")
# ax.set_ylim(bottom=1)
ax.set_ylabel("# positive cells\nper 10k cells", fontsize=fontsize, color=label_color)
# ax.set_xlabel("Timepoint", fontsize=fontsize)
# ax.legend(loc="upper left", fontsize=fontsize)

# ax.set_xticks(x, samples, rotation=0, ha="center")

ax.tick_params(axis="both", labelsize=fontsize, colors=label_color)
# ax.set_title(
#     f"Number of Zaire ebolavirus (ZEBOV) positive cells over time",
#     fontsize=fontsize + 2,
# )

ax.grid(True, which="both", color=grid_color, ls="--", lw=1)
ax.set_axisbelow(True)
ax.xaxis.grid(False)

# Plot number of distinct sOTUS detected
ax = axs[1]

all_species_nums = [] # Collect all vector for significance testing
x = np.arange(len(samples))
for i, cluster in enumerate(samples):
    # Get number of virus species with non-zero counts per cell in cluster
    array = np.array(palmdb_adata[(palmdb_adata.obs["celltype"].notnull()) & (palmdb_adata.obs['dpi_grouped']==cluster), ].X.todense()) # palmdb_adata.var["v_type"] == "macaca_only" ADD AS SUPPLEMENT TO SHOW THAT # VIRUSES INCREASES?
    species_nums = (array > 0).sum(1)
    mean = np.mean(species_nums)

    v = ax.violinplot(species_nums, [x[i]], widths=width, vert=True, showmedians=False, showextrema=False)
    for pcidx, pc in enumerate(v["bodies"]):
        pc.set_facecolor(background)
        pc.set_edgecolor(label_color)
        pc.set_alpha(1)
    ax.scatter([x[i]]*len(species_nums) + rand_jitter([x[i]]*len(species_nums), 0.3), species_nums, alpha=0.25, color=grid_color, edgecolor="grey")
    ax.scatter(x[i], mean, s=100, color=label_color)

    all_species_nums.append(species_nums)

ax.set_yscale("symlog")
# ax.set_ylim(bottom=0)
ax.set_xticks(x, labels)
ax.tick_params(axis='both', labelsize=fontsize-2, colors=label_color)
ax.set_ylabel("# distinct sOTUs", fontsize=fontsize, color=label_color)
ax.set_xlabel("Timepoint", fontsize=fontsize, color=label_color)
ax.margins(y=0.01)

ax.grid(True, which="both", color=grid_color, ls="--", lw=1)
ax.set_axisbelow(True)
ax.xaxis.grid(False)

for ax in axs:
    ax.spines['bottom'].set_color(label_color)
    ax.spines['top'].set_color(label_color)
    ax.spines['right'].set_color(label_color)
    ax.spines['left'].set_color(label_color)

# plt.tight_layout()
plt.subplots_adjust(hspace=0.08)

plt.savefig("ebola_counts_over_time.png", dpi=300, bbox_inches="tight")

fig.show()

In [ ]:
fig, axs = plt.subplots(figsize=(9, 9), nrows=2, sharex=True)
fontsize = 18
width = 0.6
color = "#003049" # Change to #b266b3 for black bkg
background = "white"
label_color = "black"
grid_color = "lightgrey"

celltypes = [
    "Monocytes",
    "B cells",
    "T cells",
    "Immature neutrophils",
    "Natural killer",
    # "Undefined",
    "MDCK"
]

labels= [
    "Monocytes",
    "B cells",
    "T cells",
    "Immature\nneutrophils",
    "Natural\nkiller",
    # "Undefined",
    "MDCK"
]

# Plot number of ZEBOV counts per celltype
ax=axs[0]

target_ids = id2tax[id2tax["species"].str.contains("Zaire ebolavirus")]["rep_ID"].unique()

cellnum = []
total_num_cells = []
for ct in celltypes:
    if ct == "MDCK":
        count = palmdb_adata[palmdb_adata.obs["species"] == "canis_lupus_familiaris", palmdb_adata.var.index.isin(target_ids)].X.sum()
        total = len(palmdb_adata.obs[palmdb_adata.obs["species"] == "canis_lupus_familiaris"])
        cellnum.append((count / total) * 10000)
        total_num_cells.append(total)
    else:
        count = palmdb_adata[palmdb_adata.obs["celltype"] == ct, palmdb_adata.var.index.isin(target_ids)].X.sum()
        total = len(palmdb_adata.obs[palmdb_adata.obs["celltype"] == ct])
        cellnum.append((count / total) * 10000)
        total_num_cells.append(total)

x = np.arange(len(celltypes))

ax.bar(
    x,
    cellnum,
    width=width,
    color=color,
    alpha=1,
    edgecolor=label_color,
)

# Add total number of cells per condition
for x_idx, nc in enumerate(total_num_cells):
    ax.text(x_idx, 115, "{:,}".format(nc), color=label_color, fontsize=fontsize, va="bottom", ha="center")

# ax.set_yscale("log")
ax.set_ylabel("# positive cells\n per 10k cells", fontsize=fontsize, color=label_color)
# ax.set_xlabel("Celltype", fontsize=fontsize)
# ax.legend(loc="upper left", fontsize=fontsize)

# ax.set_xticks(x, labels, rotation=0, ha="center")

ax.tick_params(axis="both", labelsize=fontsize, colors=label_color)
# ax.set_title(f"Number of Zaire ebolavirus (ZEBOV) positive cells per celltype", fontsize=fontsize)

ax.grid(True, which="both", color=grid_color, ls="--", lw=1)
ax.set_axisbelow(True)
ax.xaxis.grid(False)

# Plot number of distinct sOTUS detected
ax=axs[1]

x = np.arange(len(celltypes))
for i, cluster in enumerate(celltypes):
    # Get number of virus species with non-zero counts per cell in cluster
    if cluster == "MDCK":
        array = np.array(palmdb_adata[palmdb_adata.obs['species']=="canis_lupus_familiaris", :].X.todense()) # palmdb_adata.var["v_type"] == "canis_lupus_familiaris"
    else:
        array = np.array(palmdb_adata[palmdb_adata.obs['celltype']==cluster, :].X.todense())  # palmdb_adata.var["v_type"] == "macaca_only"
    species_nums = (array > 0).sum(1)
    mean = np.mean(species_nums)

    v = ax.violinplot(species_nums, [x[i]], widths=width, vert=True, showmedians=False, showextrema=False)
    for pcidx, pc in enumerate(v["bodies"]):
        pc.set_facecolor(background)
        pc.set_edgecolor(label_color)
        pc.set_alpha(1)
    ax.scatter([x[i]]*len(species_nums) + rand_jitter([x[i]]*len(species_nums), 0.3), species_nums, alpha=0.25, color=grid_color, edgecolor="grey")
    ax.scatter(x[i], mean, s=100, color=label_color)

ax.set_yscale("symlog")
ax.set_ylim(bottom=0)
ax.set_xticks(x, labels)
ax.tick_params(axis='both', labelsize=fontsize-2, colors=label_color)
ax.set_ylabel("# distinct sOTUs", fontsize=fontsize, color=label_color)
ax.set_xlabel("Celltype", fontsize=fontsize, color=label_color)
ax.margins(y=0.01)

# ax.set_title(f"Number of distinct sOTUs detected per celltype", fontsize=fontsize)

ax.grid(True, which="both", color=grid_color, ls="--", lw=1)
ax.set_axisbelow(True)
ax.xaxis.grid(False)

for ax in axs:
    ax.spines['bottom'].set_color(label_color)
    ax.spines['top'].set_color(label_color)
    ax.spines['right'].set_color(label_color)
    ax.spines['left'].set_color(label_color)

# plt.tight_layout()
plt.subplots_adjust(hspace=0.08)

plt.savefig("ebola_counts_per_celltype.png", dpi=300, bbox_inches="tight")

fig.show()

___

# Plot number of virus species per celltype per timepoint

Get normalized fractions of timepoint per celltype cluster:

In [ ]:
# Compute fraction of cells occupied per time point for each celltype cluster
df_normalized = pd.DataFrame()

adata = host_adata.copy()
groups = "celltype_clusters"
fractions = "dpi_clean_merged"
group_order = [
    "B cells 1",
    "B cells 2",
    "B cells 3",
    "B cells 4",
    "B cells 5",
    "B cells 6",
    "B cells 7",
    "T cells 1",
    "T cells 2",
    "T cells 3",
    "T cells 4",
    "T cells 5",
    "Natural killer 1",
    "Monocytes 1",
    "Monocytes 2",
    "Immature neutrophils 1",
    # "Undefined 1", # Only has 12 cells
    "Undefined 2",
    "Undefined 3"
]
group_order.reverse()

# Normalize each celltype count to total number of cells in that fraction by dividing by total
df_normalized = (
    (
        adata.obs.groupby(groups)[fractions].value_counts()
        / adata.obs.groupby(fractions)["species"].count()
    )
    .unstack()
    .fillna(0)
)

# Convert columns to string
df_normalized.columns = df_normalized.columns.astype(str)
# Get total normalized count
df_normalized["total_normalized_count"] = df_normalized.sum(axis=1).values

# Compute fraction of normalized total count for each sample
for sample in adata.obs[fractions].unique():
    df_normalized[f"{sample}_fraction"] = (
        df_normalized[sample] / df_normalized["total_normalized_count"]
    ).values

# Get total cellcount
df_normalized["total_cellcount"] = adata.obs.groupby(groups).size().values.astype(int)

# Sort data
if group_order:
    df_normalized = df_normalized.reindex(group_order)
else:
    df_normalized = df_normalized.sort_values(
        list(df_normalized.filter(like="_fraction").columns.values)
    )

In [ ]:
fig, axs = plt.subplots(figsize=(13, 20), ncols=2, sharey=True, gridspec_kw={'width_ratios': [2, 1]})

# Plot host celltype fractions
ax = axs[0]
width = 0.75
alpha = 1
fontsize = 22

colors=["#b8dd5a", "#64aa37", "#3e8938", "thistle", "rebeccapurple", "gold", "orange", "#e06237", "#b2172b", "#7d0d0f"]
frac_order=['-30d', '-4d', '0d', '4h', '24h', '3d', '4d', '5d', '6d', '7d/8d']

clusters = df_normalized.index.values
cellcounts = df_normalized["total_cellcount"].values

previous_samples = 0
for c_idx, sample in enumerate(frac_order):
    ax.barh(
        clusters,
        df_normalized[f"{sample}_fraction"].values,
        width,
        left=previous_samples,
        color=colors[c_idx],
        label=sample,
        alpha=alpha,
    )
    previous_samples = previous_samples + df_normalized[f"{sample}_fraction"].values

# Get y tick labels and split long label
labels = []
for label in clusters:
    if label == 'Immature neutrophils 1':
        labels.append('Immature\nneutrophils 1')
    else:
        labels.append(label)
ax.set_yticklabels(labels, fontsize=fontsize)

# ax.legend(bbox_to_anchor=(-0.25, 1.007), fontsize=fontsize)
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.07), ncol=5, fontsize=fontsize, title="Time point", title_fontsize=fontsize+2)

# ax.set_title(title, y=1.0, pad=20, fontsize=fontsize + 2)
ax.set_xlabel("Fraction of host cells", fontsize=fontsize+2)
ax.set_ylabel("Leiden cluster", fontsize=fontsize+2)
ax.tick_params(axis="both", labelsize=fontsize)

ax.margins(x=0.3, y=0.01)
ax.grid(False)


# Plot total number of virus species per celltype cluster
ax = axs[1]

x = np.arange(len(group_order))
for i, cluster in enumerate(group_order):
    # Get number of virus species per cell in cluster
    array = np.array(palmdb_adata[palmdb_adata.obs['celltype_clusters']==cluster, :].X.todense())
    species_nums = (array.astype(int) != 0).sum(1)
    mean = np.mean(species_nums)

    v = ax.violinplot(species_nums, [x[i]], widths=width, vert=False, showmedians=False, showextrema=False)
    for pcidx, pc in enumerate(v["bodies"]):
        pc.set_facecolor("white")
        pc.set_edgecolor("black")
        pc.set_alpha(1)
    ax.scatter(species_nums, [x[i]]*len(species_nums) + rand_jitter([x[i]]*len(species_nums), 0.3), alpha=0.25, edgecolor="grey", color="lightgrey")
    ax.scatter(mean, x[i], s=125, color="black")

# Add total cell count next to each bar
for index, value in enumerate(cellcounts):
    ax.text(x=130, y=index, s="{:,}".format(value), size=fontsize, ha="left", va="center")

ax.set_xscale("symlog")

# ax.set_xlim(left=0, right=100)
# ax.set_yticks(x, clusters)
ax.tick_params(axis='both', labelsize=fontsize+2)
ax.set_xlabel("# distinct sOTUs", fontsize=fontsize+2)
ax.margins(y=0.01)

ax.grid(True, which="both", color="lightgray", ls="--", lw=1)
ax.set_axisbelow(True)
ax.yaxis.grid(False)

plt.subplots_adjust(wspace=0.05)

plt.savefig(
    f"host_celltype_fractions_virus_species_counts.png",
    dpi=300,
    bbox_inches="tight",
)

fig.show()